In [1]:
from flask import Flask, request
from flask.json import jsonify
from pprint import pprint
import pandas as pd

from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy import inspect

import pymongo

from pyproj import Proj, transform

In [3]:
# This function adds the correct header name based on whether it was census year or not & also adds new column 'Year'
def create_basic_df_for_that_year(df, year, is_census):
    header = ["Street Address","Full Name","Race","Tenure","Census Name","Census Race"]
    # If there is No census info available for that year then make it explicitely blank data - so that 
    # the information across all the years is consistent
    
    if(is_census == 0):
        df["Census Name"] = ""
        df["Census Race"] = ""
    
    df.columns = header
    df["Year"] = year
    # handling inconsistency of Street address across multiple csv files.
    df["Street Address"] = df["Street Address"].str.replace("Street","St", regex=False) 
    return df

In [8]:
def create_additional_info_for_that_year(df, year):
    street_add = df["House No"].astype(str) + " " + df["Street Name"].astype(str)
    full_name = df["GivenName"].astype(str) + " " + df["LastName"].astype(str)

    df_new = df.loc[:,["Relation (4)","Color or Race (5)","Sex (6)","Age at last birthday (8)","Occupation (19)","Own or Rent (25)", "Owned free or mortgage (26)","House or Farm (27)"]]

    df_new["Street Address"] = street_add
    df_new["Name"] = full_name

    cols = df_new.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    cols = cols[-1:] + cols[:-1]

    df_new = df_new[cols] 
    df_new["Year"] = year
    
    return(df_new)

In [2]:
#### STEP :1 - Retrieve basic info for years 1900 through 1920 from CSV file
filename = "./Resources/basic info w race column_TableToExcel.csv"
df_basic_data_allyrs = pd.read_csv(filename)

In [18]:
df_basic_info_1900 = df_basic_data_allyrs.loc[:,["StNumClean","name1900","race1900","tenure1900","censusname1900","censusrace1900"]]
df_basic_info_1900 = create_basic_df_for_that_year(df_basic_info_1900, 1900, 1)

df_basic_info_1902 = df_basic_data_allyrs.loc[:,["StNumClean","name1902","race1902","tenure1902"]]
df_basic_info_1902 = create_basic_df_for_that_year(df_basic_info_1902, 1902, 0)

df_basic_info_1903 = df_basic_data_allyrs.loc[:,["StNumClean","name1903","race1903","tenure1903"]]
df_basic_info_1903 = create_basic_df_for_that_year(df_basic_info_1903, 1903, 0)

df_basic_info_1905 = df_basic_data_allyrs.loc[:,["StNumClean","name1905","race1905","tenure1905"]]
df_basic_info_1905 = create_basic_df_for_that_year(df_basic_info_1905, 1905, 0)

df_basic_info_1907 = df_basic_data_allyrs.loc[:,["StNumClean","name1907","race1907","tenure1907"]]
df_basic_info_1907 = create_basic_df_for_that_year(df_basic_info_1907, 1907, 0)

df_basic_info_1908 = df_basic_data_allyrs.loc[:,["StNumClean","name1908","race1908","tenure1908"]]
df_basic_info_1908 = create_basic_df_for_that_year(df_basic_info_1908, 1908, 0)

df_basic_info_1910 = df_basic_data_allyrs.loc[:,["StNumClean","name1910","race1910","tenure1910","censusname1910","censusrace1910"]]
df_basic_info_1910 = create_basic_df_for_that_year(df_basic_info_1910, 1910, 1)

df_basic_info_1911 = df_basic_data_allyrs.loc[:,["StNumClean","name1911","race1911","tenure1911"]]
df_basic_info_1911 = create_basic_df_for_that_year(df_basic_info_1911, 1911, 0)

df_basic_info_1912 = df_basic_data_allyrs.loc[:,["StNumClean","name1912","race1912","tenure1912"]]
df_basic_info_1912 = create_basic_df_for_that_year(df_basic_info_1912, 1912, 0)

df_basic_info_1913 = df_basic_data_allyrs.loc[:,["StNumClean","name1913","race1913","tenure1913"]]
df_basic_info_1913 = create_basic_df_for_that_year(df_basic_info_1913, 1913, 0)

df_basic_info_1915 = df_basic_data_allyrs.loc[:,["StNumClean","name1915","race1915","tenure1915"]]
df_basic_info_1915 = create_basic_df_for_that_year(df_basic_info_1915, 1915, 0)

df_basic_info_1917 = df_basic_data_allyrs.loc[:,["StNumClean","name1917","race1917","tenure1917"]]
df_basic_info_1917 = create_basic_df_for_that_year(df_basic_info_1917, 1917, 0)

df_basic_info_1918 = df_basic_data_allyrs.loc[:,["StNumClean","name1918","race1918","tenure1918"]]
df_basic_info_1918 = create_basic_df_for_that_year(df_basic_info_1918, 1918, 0)

df_basic_info_1919 = df_basic_data_allyrs.loc[:,["StNumClean","name1919","race1919","tenure1919"]]
df_basic_info_1919 = create_basic_df_for_that_year(df_basic_info_1919, 1919, 0)

df_basic_info_1920 = df_basic_data_allyrs.loc[:,["StNumClean","name1920","race1920","tenure1920","censusname1920","censusrace1920"]]
df_basic_info_1920 = create_basic_df_for_that_year(df_basic_info_1920, 1920, 1)

df_basic_info_1920.head()

,Street Address,Full Name,Race,Tenure,Census Name,Census Race,Year
0,805 Arthur St,NOT LISTED,0,0,NaN,NaN,1920
1,830 Arthur St,Clarence Dupree ©,1,1,NaN,NaN,1920
2,832 Arthur St,NOT LISTED,0,0,NaN,NaN,1920
3,834 Arthur St,Charles Madison ©,1,1,NaN,NaN,1920
4,836 Arthur St,Mattie Larshade ©,1,1,NaN,NaN,1920


In [5]:
# Initialize PyMongo to work with MongoDBs & create our OWN db to store information
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Retrieve OR Create New database and collection
db_DH = client.digitalHumanity_db
censuses_year1900_collection = db_DH.censuses_1900
censuses_year1902_collection = db_DH.censuses_1902
censuses_year1903_collection = db_DH.censuses_1903
censuses_year1905_collection = db_DH.censuses_1905
censuses_year1907_collection = db_DH.censuses_1907
censuses_year1908_collection = db_DH.censuses_1908
censuses_year1910_collection = db_DH.censuses_1910
censuses_year1911_collection = db_DH.censuses_1911
censuses_year1912_collection = db_DH.censuses_1912
censuses_year1913_collection = db_DH.censuses_1913
censuses_year1915_collection = db_DH.censuses_1915
censuses_year1917_collection = db_DH.censuses_1917
censuses_year1918_collection = db_DH.censuses_1918
censuses_year1919_collection = db_DH.censuses_1919
censuses_year1920_collection = db_DH.censuses_1920

In [6]:
# insert dataframes into respective MongoDB collections
censuses_year1900_collection.insert_many(df_basic_info_1900.to_dict('records'))
censuses_year1902_collection.insert_many(df_basic_info_1902.to_dict('records'))
censuses_year1903_collection.insert_many(df_basic_info_1903.to_dict('records'))
censuses_year1905_collection.insert_many(df_basic_info_1905.to_dict('records'))
censuses_year1907_collection.insert_many(df_basic_info_1907.to_dict('records'))
censuses_year1908_collection.insert_many(df_basic_info_1908.to_dict('records'))
censuses_year1910_collection.insert_many(df_basic_info_1910.to_dict('records'))
censuses_year1911_collection.insert_many(df_basic_info_1911.to_dict('records'))
censuses_year1912_collection.insert_many(df_basic_info_1912.to_dict('records'))
censuses_year1913_collection.insert_many(df_basic_info_1913.to_dict('records'))
censuses_year1915_collection.insert_many(df_basic_info_1915.to_dict('records'))
censuses_year1917_collection.insert_many(df_basic_info_1917.to_dict('records'))
censuses_year1918_collection.insert_many(df_basic_info_1918.to_dict('records'))
censuses_year1919_collection.insert_many(df_basic_info_1919.to_dict('records'))
censuses_year1920_collection.insert_many(df_basic_info_1920.to_dict('records'))


In [14]:
#### --- STEP :2 ----- Retrieve Occupation, Color, Age, Sex, Owned based on Street Address, 
# Year & Full Name/ Census Name
# Retrieve additional info for census years 1900, 1910 & 1920 from CSV file
filename_1900 = "./Resources/Reservation Census charting_1900.csv"
df_reservation_1900 = pd.read_csv(filename_1900)

filename_1910 = "./Resources/Reservation Census charting_1910.csv"
df_reservation_1910 = pd.read_csv(filename_1910)

filename_1920 = "./Resources/Reservation Census charting_1920.csv"
df_reservation_1920 = pd.read_csv(filename_1920)


In [15]:
df_reservation_1920.count()

Line #                         3282
House No                       3282
Street Name                    3282
LastName                       3281
GivenName                      3281
Relation (4)                   3281
Own or Rent (25)                905
Owned free or mortgage (26)      60
House or Farm (27)               14
Color or Race (5)              3282
Sex (6)                        3282
Age at last birthday (8)       3282
Occupation (19)                3275
My Notes                          6
File Name                      3282
dtype: int64

In [16]:
df_new_1900 = create_additional_info_for_that_year(df_reservation_1900, 1900)
df_new_1910 = create_additional_info_for_that_year(df_reservation_1910, 1910)
df_new_1920 = create_additional_info_for_that_year(df_reservation_1920, 1920)

In [17]:
df_new_1920

,Street Address,Name,Relation (4),Color or Race (5),Sex (6),Age at last birthday (8),Occupation (19),Own or Rent (25),Owned free or mortgage (26),House or Farm (27),Year
0,322 Rusk Avenue,Hans Halner,Head,W,M,32,Blacksmith At Shops,NaN,NaN,House,1920
1,322 Rusk Avenue,Teresa Halner,Wife,W,F,27,Dress Maker At Home,NaN,NaN,NaN,1920
2,322 Rusk Avenue,Louis Halner,Son,W,M,9,None,NaN,NaN,NaN,1920
3,322 Rusk Avenue,Riehard Halner,Son,W,M,5 and 0 months,None,NaN,NaN,NaN,1920
4,322 Rusk Avenue,Ben Halner,Son,W,M,3,None,NaN,NaN,NaN,1920
5,322 Rusk Avenue,Martin Halner,Son,W,M,2 months,None,NaN,NaN,NaN,1920
6,320 Rusk Avenue,George Bell,Head,B,M,35,Longshoreman At Port,Rent,NaN,House,1920
7,320 Rusk Avenue,Bullia Bell,Wife,Mu,F,26,None,NaN,NaN,NaN,1920
8,318 Rusk Avenue,Carrie Fleming,Head,Mu,F,51,Washing At Home,Rent,NaN,House,1920
9,316 Rusk Avenue,Arthur H. Nelson,Head,M,M,26,Cook At Train,Rent,NaN,House,1920


In [11]:
# Update the existing documents within a collection with new additional info, if NOT existing then create new document
# under the given collection for all 3 census years

for index, row in df_new_1900.iterrows():
    censuses_year1900_collection.update_many({'Year': row[-1], 'Street Address': row[0], 'Census Name': row[1]}, {"$set": row.to_dict()}, upsert=True)

for index, row in df_new_1910.iterrows():
    censuses_year1910_collection.update_many({'Year': row[-1], 'Street Address': row[0], 'Census Name': row[1]}, {"$set": row.to_dict()}, upsert=True)
    
for index, row in df_new_1920.iterrows():
    censuses_year1920_collection.update_many({'Year': row[-1], 'Street Address': row[0], 'Census Name': row[1]}, {"$set": row.to_dict()}, upsert=True)    


In [56]:
#### --- STEP :3 ----- Retrieve info of X & Y (State Plane coordinates from CSV file) based on Street Address, 
# Year & Full Name/ Census Name
# ------------------------------------------------------------------------------------------------
filename_latlong = "./Resources/LatLong_Info.csv"
df_latlong = pd.read_csv(filename_latlong)
# handling inconsistency of Street address across multiple csv files.
df_latlong["Street Address"] = df_latlong["Street Address"].str.replace("Street","St", regex=False) 
df_latlong.head()

,Latitude,Longitude,Street Address
0,-95.377021,29.758413,805 Arthur St
1,-95.377083,29.758487,832 Arthur St
2,-95.377086,29.758785,840 Arthur St
3,-95.377083,29.758413,830 Arthur St
4,-95.377027,29.759003,841 Arthur St


In [59]:
def update_db_document_for_that_particular_coll(df, year, db_coll_that_year):
    for index, row in df.iterrows():
        db_coll_that_year.update_many({'Year': year, 'Street Address': row[2]}, {"$set": row.to_dict()}, upsert=True)


In [62]:
update_db_document_for_that_particular_coll(df_latlong, 1900, censuses_year1900_collection)
update_db_document_for_that_particular_coll(df_latlong, 1902, censuses_year1902_collection)
update_db_document_for_that_particular_coll(df_latlong, 1903, censuses_year1903_collection)
update_db_document_for_that_particular_coll(df_latlong, 1905, censuses_year1905_collection)
update_db_document_for_that_particular_coll(df_latlong, 1907, censuses_year1907_collection)
update_db_document_for_that_particular_coll(df_latlong, 1908, censuses_year1908_collection)
update_db_document_for_that_particular_coll(df_latlong, 1910, censuses_year1910_collection)
update_db_document_for_that_particular_coll(df_latlong, 1911, censuses_year1911_collection)
update_db_document_for_that_particular_coll(df_latlong, 1912, censuses_year1912_collection)
update_db_document_for_that_particular_coll(df_latlong, 1913, censuses_year1913_collection)
update_db_document_for_that_particular_coll(df_latlong, 1915, censuses_year1915_collection)
update_db_document_for_that_particular_coll(df_latlong, 1917, censuses_year1917_collection)
update_db_document_for_that_particular_coll(df_latlong, 1918, censuses_year1918_collection)
update_db_document_for_that_particular_coll(df_latlong, 1919, censuses_year1919_collection)
update_db_document_for_that_particular_coll(df_latlong, 1920, censuses_year1920_collection)